<a href="https://colab.research.google.com/github/Serena-G-LEE/25-2_Project/blob/main/AI_cross_department_relationship_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 셀 1: 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# 셀 2: 라이브러리 임포트 및 기본 경로 설정
import os
import glob
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 선생님 구글 드라이브 내 프로젝트 폴더 (필요하면 여기만 수정하시면 됩니다)
BASE_DIR = "/content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트"

# 연도별 폴더 이름 매핑
# 2011년 예시는 주어진 그대로 사용했고,
# 나머지 연도는 실제 폴더 이름에 맞게 선생님께서 직접 채워 넣으시면 됩니다.
YEAR_FOLDER_MAP = {
    2011: "openalex_data_cleaning_2011_with_authors",
    2012: "openalex_data_cleaning_2012_with_authors",
    2013: "openalex_data_cleaning_2013_with_authors",
    2014: "openalex_data_cleaning_2014_with_authors",
    2015: "openalex_data_cleaning_2015_with_authors",
    2016: "openalex_data_cleaning_2016_with_authors",
    2017: "openalex_data_cleaning_2017_with_authors",
    2018: "openalex_data_cleaning_2018_with_authors",
    2019: "openalex_data_cleaning_2019_with_authors",
    2020: "openalex_data_cleaning_2020_with_authors",
    2021: "openalex_data_cleaning_2021_with_authors",
    2022: "openalex_data_cleaning_2022_with_authors",
    2023: "openalex_data_cleaning_2023_with_authors",
    2024: "openalex_data_cleaning_2024_with_authors",
    2025: "openalex_data_cleaning_2025_with_authors",
}

# 텍스트 결합에 사용할 컬럼
# 정제된 제목/초록 컬럼을 사용합니다.
TEXT_COLUMNS = ["title_clean", "abstract_clean"]

print("BASE_DIR:", BASE_DIR)
print("YEAR_FOLDER_MAP:", YEAR_FOLDER_MAP)
print("TEXT_COLUMNS:", TEXT_COLUMNS)


BASE_DIR: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트
YEAR_FOLDER_MAP: {2011: 'openalex_data_cleaning_2011_with_authors'}
TEXT_COLUMNS: ['title_clean', 'abstract_clean']


In [3]:
# 셀 3: 예시 파일 열어서 컬럼 구조 확인

sample_year = 2011  # 다른 연도를 보고 싶으면 이 값만 바꾸세요.

if sample_year not in YEAR_FOLDER_MAP:
    raise ValueError(f"{sample_year}년이 YEAR_FOLDER_MAP에 없습니다. YEAR_FOLDER_MAP을 먼저 채워 주세요.")

year_folder = YEAR_FOLDER_MAP[sample_year]
folder_path = os.path.join(BASE_DIR, year_folder)

print("폴더 경로:", folder_path)
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
print(f"{sample_year}년 폴더 내 CSV 파일 수:", len(csv_files))
for f in csv_files:
    print(" -", os.path.basename(f))

# ALL 파일 패턴에 맞는 파일을 읽어서 컬럼 확인
all_pattern = os.path.join(folder_path, f"works_top10000_{sample_year}_ALL.cleaned.csv")
all_files = glob.glob(all_pattern)

if len(all_files) > 0:
    print("\n예시로 읽는 파일:", all_files[0])
    df_sample = pd.read_csv(all_files[0])
    print("\n샘플 파일 컬럼:")
    print(df_sample.columns)
else:
    print("\nworks_top10000_YYYY_ALL.cleaned.csv 패턴에 맞는 파일을 찾지 못했습니다.")
    print("연도 또는 파일 이름 패턴을 직접 확인해 주세요.")


폴더 경로: /content/drive/MyDrive/6. 덕성여자대학교/25학년도 2학기/비정형데이터분석/프로젝트/openalex_data_cleaning_2011_with_authors
2011년 폴더 내 CSV 파일 수: 25
 - institutions_mapping.cleaned.csv
 - works_top10000_2011_ALL.cleaned.csv
 - works_top10000_2011_Chonnam_National_University.cleaned.csv
 - works_top10000_2011_Chung-Ang_University.cleaned.csv
 - works_top10000_2011_Chungbuk_National_University.cleaned.csv
 - works_top10000_2011_Chungnam_National_University.cleaned.csv
 - works_top10000_2011_Daegu_Gyeongbuk_Institute_of_Science_and_Technology.cleaned.csv
 - works_top10000_2011_Duksung_Women's_University.cleaned.csv
 - works_top10000_2011_Gwangju_Institute_of_Science_and_Technology.cleaned.csv
 - works_top10000_2011_Gyeongsang_National_University.cleaned.csv
 - works_top10000_2011_Hanyang_University.cleaned.csv
 - works_top10000_2011_Jeju_National_University.cleaned.csv
 - works_top10000_2011_Jeonbuk_National_University.cleaned.csv
 - works_top10000_2011_Kangwon_National_University.cleaned.csv
 - works_top10

In [4]:
# 셀 4: 인공지능 관련 키워드 정의 (소문자)

AI_KEYWORDS = [
    # 일반 인공지능, 머신러닝
    "artificial intelligence",
    "intelligent system",
    "intelligent systems",
    "machine learning",
    "statistical learning",
    "deep learning",
    "representation learning",
    "feature learning",
    "data mining",
    "data-driven",
    "predictive model",
    "predictive modeling",
    "pattern recognition",
    "knowledge discovery",

    # 신경망 및 딥러닝 구조
    "neural network",
    "neural networks",
    "artificial neural network",
    "deep neural network",
    "convolutional neural network",
    "cnn",
    "recurrent neural network",
    "rnn",
    "long short term memory",
    "long short-term memory",
    "lstm",
    "gated recurrent unit",
    "gru",
    "transformer",
    "transformer-based",
    "attention mechanism",
    "self-attention",
    "graph neural network",
    "gnn",
    "graph convolutional network",
    "gcn",
    "autoencoder",
    "variational autoencoder",
    "vae",
    "generative adversarial network",
    "gan",

    # 고전적 머신러닝 알고리즘
    "support vector machine",
    "svm",
    "random forest",
    "decision tree",
    "decision trees",
    "gradient boosting",
    "xgboost",
    "lightgbm",
    "k-nearest neighbor",
    "k-nearest neighbours",
    "knn",
    "naive bayes",
    "bayesian classifier",
    "bayesian network",
    "logistic regression",
    "linear classifier",
    "clustering",
    "k-means",
    "hierarchical clustering",
    "principal component analysis",
    "pca",

    # 강화학습
    "reinforcement learning",
    "rl",
    "q-learning",
    "deep q network",
    "dqn",
    "policy gradient",
    "actor critic",
    "actor-critic",

    # 자연어처리 및 시계열
    "natural language processing",
    "nlp",
    "language model",
    "large language model",
    "llm",
    "text classification",
    "sentiment analysis",
    "machine translation",
    "question answering",
    "sequence model",
    "sequential model",
    "time series forecasting",
    "time-series forecasting",
    "speech recognition",
    "automatic speech recognition",
    "asr",

    # 컴퓨터 비전
    "computer vision",
    "image classification",
    "object detection",
    "semantic segmentation",
    "instance segmentation",
    "pose estimation",
    "face recognition",
    "facial recognition",
    "scene understanding",
    "visual tracking",

    # 기타 표현
    "hyperparameter tuning",
    "hyper-parameter optimization",
    "meta-learning",
    "transfer learning",
    "domain adaptation",
    "few-shot learning",
    "zero-shot learning",
    "self-supervised learning",
    "semi-supervised learning",
    "anomaly detection",
    "outlier detection",
    "fault diagnosis",
    "predictive maintenance",
]

print("AI 키워드 개수:", len(AI_KEYWORDS))


AI 키워드 개수: 108


In [5]:
# 셀 5: 공학 소분류별 키워드 정의 (소문자)

ENGINEERING_SUBFIELDS = {
    "arch_building": {
        "mid_ko": "건축",
        "sub_ko": "건축·설비공학",
        "keywords": [
            "building engineering",
            "building services",
            "building system",
            "building systems",
            "hvac",
            "heating ventilation and air conditioning",
            "ventilation",
            "air conditioning",
            "thermal comfort",
            "indoor environment",
            "indoor air quality",
            "building energy",
            "building performance",
            "building envelope",
            "building retrofit",
        ],
    },
    "arch_architecture": {
        "mid_ko": "건축",
        "sub_ko": "건축학",
        "keywords": [
            "architecture",
            "architectural design",
            "architectural engineering",
            "building design",
            "building layout",
            "floor plan",
            "architectural planning",
            "housing design",
        ],
    },
    "arch_landscape": {
        "mid_ko": "건축",
        "sub_ko": "조경학",
        "keywords": [
            "landscape architecture",
            "landscape design",
            "urban landscape",
            "green space",
            "green spaces",
            "green infrastructure",
            "park design",
            "open space",
            "recreational space",
        ],
    },
    "civil_civil": {
        "mid_ko": "토목·도시",
        "sub_ko": "토목공학",
        "keywords": [
            "civil engineering",
            "structural engineering",
            "geotechnical engineering",
            "bridge",
            "bridges",
            "tunnel",
            "tunnels",
            "dam",
            "dams",
            "foundation",
            "soil mechanics",
            "soil stabilization",
            "slope stability",
            "pavement",
            "pavement design",
            "highway engineering",
        ],
    },
    "civil_urban": {
        "mid_ko": "토목·도시",
        "sub_ko": "도시공학",
        "keywords": [
            "urban planning",
            "urban design",
            "urban engineering",
            "land use",
            "land-use",
            "zoning",
            "urban development",
            "smart city",
            "smart cities",
            "urban regeneration",
            "urban renewal",
            "transportation planning",
            "urban mobility",
            "urban sprawl",
        ],
    },
    "transport_ground": {
        "mid_ko": "교통·운송",
        "sub_ko": "지상교통공학",
        "keywords": [
            "transportation engineering",
            "traffic engineering",
            "traffic flow",
            "traffic congestion",
            "traffic safety",
            "traffic signal",
            "intelligent transportation system",
            "its",
            "public transport",
            "public transportation",
            "bus system",
            "railway",
            "rail transport",
        ],
    },
    "transport_aero": {
        "mid_ko": "교통·운송",
        "sub_ko": "항공학",
        "keywords": [
            "aerospace engineering",
            "aeronautical engineering",
            "aircraft design",
            "aircraft control",
            "flight control",
            "flight dynamics",
            "aerodynamics",
            "propulsion system",
            "jet engine",
            "aeroelasticity",
        ],
    },
    "transport_marine": {
        "mid_ko": "교통·운송",
        "sub_ko": "해양공학",
        "keywords": [
            "ocean engineering",
            "marine engineering",
            "naval architecture",
            "ship design",
            "ship hydrodynamics",
            "offshore structure",
            "offshore platform",
            "marine structure",
            "subsea structure",
            "coastal engineering",
        ],
    },
    "mech_mechanical": {
        "mid_ko": "기계·금속",
        "sub_ko": "기계공학",
        "keywords": [
            "mechanical engineering",
            "mechanical design",
            "dynamics",
            "kinematics",
            "mechanism design",
            "mechatronics",
            "robot",
            "robots",
            "robotics",
            "robotic system",
            "robotic arm",
            "fluid mechanics",
            "heat transfer",
            "thermofluid",
            "thermodynamics",
            "vibration analysis",
            "structural vibration",
        ],
    },
    "mech_metal": {
        "mid_ko": "기계·금속",
        "sub_ko": "금속공학",
        "keywords": [
            "metallurgy",
            "metallurgical engineering",
            "metallurgical",
            "metal materials",
            "metallic material",
            "metallic alloy",
            "steel",
            "stainless steel",
            "aluminum alloy",
            "titanium alloy",
            "heat treatment",
            "microstructure",
            "phase transformation",
        ],
    },
    "mech_automotive": {
        "mid_ko": "기계·금속",
        "sub_ko": "자동차공학",
        "keywords": [
            "automotive engineering",
            "vehicle engineering",
            "vehicle dynamics",
            "electric vehicle",
            "hybrid vehicle",
            "autonomous vehicle",
            "self-driving car",
            "powertrain",
            "engine performance",
            "emission control",
            "vehicle control",
            "chassis",
            "suspension",
            "braking system",
        ],
    },
    "elec_electrical": {
        "mid_ko": "전기·전자",
        "sub_ko": "전기공학",
        "keywords": [
            "electrical engineering",
            "power system",
            "power systems",
            "power grid",
            "smart grid",
            "distribution system",
            "transmission line",
            "electric machine",
            "electric motor",
            "generator",
            "transformer",
            "power quality",
            "renewable integration",
        ],
    },
    "elec_electronic": {
        "mid_ko": "전기·전자",
        "sub_ko": "전자공학",
        "keywords": [
            "electronics",
            "electronic engineering",
            "electronic circuit",
            "electronic circuits",
            "semiconductor device",
            "integrated circuit",
            "vlsi",
            "analog circuit",
            "digital circuit",
            "mixed-signal",
            "signal integrity",
            "microelectronics",
            "nanoelectronics",
        ],
    },
    "elec_control": {
        "mid_ko": "전기·전자",
        "sub_ko": "제어계측공학",
        "keywords": [
            "control engineering",
            "control system",
            "control systems",
            "automatic control",
            "feedback control",
            "robust control",
            "optimal control",
            "pid control",
            "state estimation",
            "sensor",
            "sensors",
            "sensing system",
            "measurement system",
            "instrumentation",
        ],
    },
    "precise_optical": {
        "mid_ko": "정밀·에너지",
        "sub_ko": "광학공학",
        "keywords": [
            "optical engineering",
            "applied optics",
            "laser system",
            "optical fiber",
            "optical communication",
            "imaging system",
            "optical sensor",
            "photonics",
            "photonic device",
        ],
    },
    "precise_energy": {
        "mid_ko": "정밀·에너지",
        "sub_ko": "에너지공학",
        "keywords": [
            "energy engineering",
            "energy system",
            "energy systems",
            "renewable energy",
            "solar energy",
            "wind energy",
            "hydro power",
            "fuel cell",
            "energy storage",
            "battery system",
            "thermal energy storage",
            "energy efficiency",
            "energy management",
        ],
    },
    "materials_semiceramic": {
        "mid_ko": "소재·재료",
        "sub_ko": "반도체·세라믹공학",
        "keywords": [
            "semiconductor engineering",
            "semiconductor processing",
            "device fabrication",
            "wafer",
            "lithography",
            "ceramic material",
            "ceramics",
            "ferroelectric",
            "piezoelectric",
        ],
    },
    "materials_textile": {
        "mid_ko": "소재·재료",
        "sub_ko": "섬유공학",
        "keywords": [
            "textile engineering",
            "textile material",
            "fiber material",
            "polymer fiber",
            "synthetic fiber",
            "natural fiber",
            "yarn",
            "weaving",
            "spinning",
        ],
    },
    "materials_advanced": {
        "mid_ko": "소재·재료",
        "sub_ko": "신소재공학",
        "keywords": [
            "advanced materials",
            "novel material",
            "functional material",
            "nanomaterial",
            "nanomaterials",
            "nanostructure",
            "composite material",
            "composites",
            "smart material",
        ],
    },
    "materials_material": {
        "mid_ko": "소재·재료",
        "sub_ko": "재료공학",
        "keywords": [
            "materials science",
            "materials engineering",
            "mechanical property",
            "mechanical properties",
            "fracture",
            "fatigue",
            "corrosion",
            "coating",
            "alloy design",
            "phase diagram",
        ],
    },
    "industrial_industrial": {
        "mid_ko": "산업",
        "sub_ko": "산업공학",
        "keywords": [
            "industrial engineering",
            "systems engineering",
            "operations research",
            "production planning",
            "scheduling",
            "supply chain",
            "supply chain management",
            "logistics",
            "inventory control",
            "quality control",
            "process optimization",
            "human factors",
            "ergonomics",
        ],
    },
    "chem_chemical": {
        "mid_ko": "화공",
        "sub_ko": "화학공학",
        "keywords": [
            "chemical engineering",
            "process engineering",
            "reactor design",
            "chemical reactor",
            "catalysis",
            "catalytic reaction",
            "distillation",
            "separation process",
            "membrane process",
            "polymerization",
            "polymer process",
            "reaction kinetics",
            "process control",
            "process optimization",
            "process simulation",
        ],
    },
}

print("소분류 개수:", len(ENGINEERING_SUBFIELDS))


소분류 개수: 22


In [6]:
# 셀 6: 전처리 및 키워드 매칭 함수 정의

def combine_text_columns(df, text_columns):
    """
    여러 텍스트 컬럼(예: 제목, 초록)을 하나의 문자열로 합쳐서 소문자로 변환합니다.
    """
    text = pd.Series("", index=df.index)
    for col in text_columns:
        if col in df.columns:
            text = text + " " + df[col].fillna("").astype(str)
    return text.str.lower()

def make_or_pattern(keywords):
    """
    키워드 리스트를 정규식 OR 패턴으로 변환합니다.
    """
    escaped = [re.escape(k) for k in keywords if k]
    if not escaped:
        return None
    pattern = "(" + "|".join(escaped) + ")"
    return pattern

def make_ai_mask(text_series):
    """
    AI_KEYWORDS 또는 'ai'라는 단어를 포함하는지 여부를 반환합니다.
    """
    pattern = make_or_pattern(AI_KEYWORDS)
    if pattern is None:
        base_mask = pd.Series(False, index=text_series.index)
    else:
        base_mask = text_series.str.contains(pattern, case=False, regex=True, na=False)

    # 'ai' 단어 자체 (약어)도 별도로 잡습니다.
    ai_word_mask = text_series.str.contains(r"\bai\b", case=False, regex=True, na=False)

    return base_mask | ai_word_mask

# 공학 소분류별 정규식 패턴 미리 생성
SUBFIELD_PATTERNS = {}
for key, info in ENGINEERING_SUBFIELDS.items():
    pattern = make_or_pattern(info["keywords"])
    SUBFIELD_PATTERNS[key] = pattern

print("SUBFIELD_PATTERNS 개수:", len(SUBFIELD_PATTERNS))


SUBFIELD_PATTERNS 개수: 22


In [7]:
# 셀 7: 파일 이름 파싱 + 연도/학교별 데이터 로딩

def parse_year_and_university_from_filename(path):
    """
    파일 이름에서 연도와 학교 이름을 추출합니다.
    예: works_top10000_2011_Seoul_National_University.cleaned.csv
         -> year=2011, university='Seoul_National_University'
    """
    base = os.path.basename(path)
    name = os.path.splitext(base)[0]  # .csv 제거 (있다면)

    # .cleaned로 끝나면 제거
    if name.endswith(".cleaned"):
        name = name[: -len(".cleaned")]

    prefix = "works_top10000_"
    if name.startswith(prefix):
        name = name[len(prefix):]

    parts = name.split("_", 1)
    if len(parts) != 2:
        return None, None
    year_str, uni_str = parts
    try:
        year = int(year_str)
    except ValueError:
        return None, None
    return year, uni_str

def load_all_university_data(base_dir, year_folder_map):
    """
    연도별 폴더에서 학교별 파일을 모두 읽어 하나의 DataFrame으로 합칩니다.
    ALL 파일은 제외하고, 학교별 파일만 사용합니다.
    """
    dfs = []

    for year, folder_name in year_folder_map.items():
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.isdir(folder_path):
            print(f"[경고] {year}년 폴더가 존재하지 않습니다: {folder_path}")
            continue

        pattern = os.path.join(folder_path, f"works_top10000_{year}_*.cleaned*")
        files = glob.glob(pattern)

        if not files:
            print(f"[경고] {year}년에서 매칭되는 파일이 없습니다: {pattern}")
            continue

        print(f"{year}년: 파일 {len(files)}개 발견")

        for path in files:
            y, uni_str = parse_year_and_university_from_filename(path)
            if y is None or uni_str is None:
                print("  [무시] 파일 이름에서 연도/학교를 파싱하지 못했습니다:", os.path.basename(path))
                continue

            # ALL 파일은 제외
            if uni_str.upper() == "ALL":
                continue

            try:
                df = pd.read_csv(path)
            except Exception as e:
                print("  [무시] CSV를 읽는 중 오류:", os.path.basename(path), "오류:", e)
                continue

            df["year_from_filename"] = y
            df["university_from_filename"] = uni_str.replace("_", " ")
            dfs.append(df)

    if dfs:
        df_all = pd.concat(dfs, ignore_index=True)
    else:
        df_all = pd.DataFrame()

    return df_all

df_all = load_all_university_data(BASE_DIR, YEAR_FOLDER_MAP)
print("전체 행 수:", len(df_all))

if not df_all.empty:
    print(df_all[["year_from_filename", "university_from_filename"]].drop_duplicates().head())


2011년: 파일 24개 발견
전체 행 수: 43156
      year_from_filename                           university_from_filename
0                   2011                        Chonnam National University
1782                2011                               Chung-Ang University
3272                2011                       Chungbuk National University
4642                2011                       Chungnam National University
6608                2011  Daegu Gyeongbuk Institute of Science and Techn...


In [8]:
# 셀 8: 텍스트 결합 및 AI 관련 논문 필터링

if df_all.empty:
    print("df_all이 비어 있습니다. YEAR_FOLDER_MAP과 파일 경로를 먼저 확인해 주세요.")
else:
    # 제목 + 초록 등을 하나의 텍스트로 합치기
    df_all = df_all.copy()
    df_all["text_all"] = combine_text_columns(df_all, TEXT_COLUMNS)

    print("text_all 예시:")
    print(df_all["text_all"].head())

    # AI 관련 키워드 포함 여부
    df_all["is_ai"] = make_ai_mask(df_all["text_all"])
    df_ai = df_all[df_all["is_ai"]].copy()

    print("AI 관련 논문 수:", len(df_ai))
    print(df_ai[["year_from_filename", "university_from_filename"]].head())


text_all 예시:
0     high cleavage efficiency 2 peptide derived po...
1     aba signal transduction crossroad biotic abio...
2       polyacrylonitrile nanofibers state art review 
3     six month versus 12 month dual antiplatelet t...
4     myb 96 transcription factor regulates cuticul...
Name: text_all, dtype: object


/tmp/ipython-input-3295042140.py:31: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  base_mask = text_series.str.contains(pattern, case=False, regex=True, na=False)


AI 관련 논문 수: 9502
    year_from_filename     university_from_filename
3                 2011  Chonnam National University
8                 2011  Chonnam National University
10                2011  Chonnam National University
13                2011  Chonnam National University
16                2011  Chonnam National University


In [9]:
# 셀 9: 공학 소분류 키워드 매칭 및 AI+공학 논문 추출

if df_all.empty:
    print("df_all이 비어 있습니다. 먼저 데이터 로딩과 AI 필터링을 확인해 주세요.")
elif "is_ai" not in df_all.columns:
    print("df_all에 is_ai가 없습니다. 셀 8을 먼저 실행해 주세요.")
else:
    df_ai = df_all[df_all["is_ai"]].copy()

    ai_eng_dfs = []

    for sub_key, info in ENGINEERING_SUBFIELDS.items():
        pattern = SUBFIELD_PATTERNS.get(sub_key)
        if not pattern:
            continue

        mask_sub = df_ai["text_all"].str.contains(pattern, case=False, regex=True, na=False)
        sub_df = df_ai[mask_sub].copy()

        if sub_df.empty:
            continue

        sub_df["subfield_key"] = sub_key
        sub_df["subfield_mid_ko"] = info["mid_ko"]
        sub_df["subfield_sub_ko"] = info["sub_ko"]

        ai_eng_dfs.append(sub_df)

    if ai_eng_dfs:
        df_ai_eng = pd.concat(ai_eng_dfs, ignore_index=True)
        print("AI + 공학 소분류에 동시에 해당하는 논문 수:", len(df_ai_eng))
        print(df_ai_eng[[
            "year_from_filename",
            "university_from_filename",
            "subfield_mid_ko",
            "subfield_sub_ko",
        ]].head())
    else:
        df_ai_eng = pd.DataFrame()
        print("현재 키워드 설정으로 매칭되는 AI+공학 논문이 없습니다. 키워드 범위를 넓혀 보셔도 됩니다.")


/tmp/ipython-input-145129724.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(pattern, case=False, regex=True, na=False)
/tmp/ipython-input-145129724.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(pattern, case=False, regex=True, na=False)
/tmp/ipython-input-145129724.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(pattern, case=False, regex=True, na=False)
/tmp/ipython-input-145129724.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_sub = df_ai["text_all"].str.contains(pattern, case=False

AI + 공학 소분류에 동시에 해당하는 논문 수: 3559
   year_from_filename      university_from_filename subfield_mid_ko  \
0                2011          Chung-Ang University              건축   
1                2011          Chung-Ang University              건축   
2                2011          Chung-Ang University              건축   
3                2011  Chungbuk National University              건축   
4                2011  Chungbuk National University              건축   

  subfield_sub_ko  
0         건축·설비공학  
1         건축·설비공학  
2         건축·설비공학  
3         건축·설비공학  
4         건축·설비공학  


In [10]:
# 셀 10: 학교별·공학 소분류별 AI 논문 수 집계 예시

if df_ai_eng.empty:
    print("df_ai_eng가 비어 있습니다. 셀 9까지의 결과를 먼저 확인해 주세요.")
else:
    # 학교별 + 공학 소분류별 논문 수
    grouped_uni_sub = (
        df_ai_eng
        .groupby(["university_from_filename", "subfield_mid_ko", "subfield_sub_ko"])
        .size()
        .reset_index(name="paper_count")
        .sort_values("paper_count", ascending=False)
    )

    print("학교별, 공학 소분류별 AI 접목 논문 수 상위 20개")
    print(grouped_uni_sub.head(20))

    # 연도별 + 공학 중분류별 추세
    grouped_year_mid = (
        df_ai_eng
        .groupby(["year_from_filename", "subfield_mid_ko"])
        .size()
        .reset_index(name="paper_count")
        .sort_values(["year_from_filename", "paper_count"], ascending=[True, False])
    )

    print("\n연도별, 공학 중분류별 AI 접목 논문 수 예시 (상위 20행)")
    print(grouped_year_mid.head(20))


학교별, 공학 소분류별 AI 접목 논문 수 상위 20개
                              university_from_filename subfield_mid_ko  \
247                          Seoul National University           교통·운송   
264                          Seoul National University           토목·도시   
314                                  Yonsei University           교통·운송   
161  Korea Advanced Institute of Science and Techno...           교통·운송   
328                                  Yonsei University           토목·도시   
260                          Seoul National University           전기·전자   
173  Korea Advanced Institute of Science and Techno...           전기·전자   
282                            Sungkyunkwan University           교통·운송   
251                          Seoul National University           기계·금속   
176  Korea Advanced Institute of Science and Techno...           토목·도시   
325                                  Yonsei University           전기·전자   
211        Pohang University of Science and Technology           교통·운송   
296    